In [26]:
%load_ext autoreload
%autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import sys

sys.path.append('../')

In [28]:
from src.models import LLMRetriever
#LLMAPIKEY = ""

#Instantiate the retriever
retriever = LLMRetriever(api_key)

# Prompt user for a landmark name
landmark_name = input("Enter the name of a landmark (e.g., 'One World Trade Center, NYC'): ")

# Fetch and display landmark information
landmark_details = retriever.fetch_landmark_details(landmark_name)
print("\n=== Landmark Information ===")
# Print the processed response with proper formatting
# Print the processed response with proper formatting
print(landmark_details.model_dump_json(indent=4))



=== Landmark Information ===
{
    "description": "The Burj Al Arab is a luxury hotel located in Dubai, United Arab Emirates. It is known for its distinctive sail-shaped design and is considered one of the most luxurious hotels in the world.",
    "architects": "Tom Wright of WKK Architects",
    "construction_year": "1999",
    "materials": "Concrete, steel, and glass",
    "interesting_facts": "The hotel stands on an artificial island, 280 meters offshore., The Burj Al Arab is the only 7-star hotel in the world.",
    "dimensions": "Height: 321 meters (1,053 ft); Number of floors: 60",
    "construction_cost": "1.5 billion USD",
    "cultural_significance": "Symbol of Dubai's engineering prowess and luxury tourism industry",
    "geographical_location": "Off the coast of Dubai, United Arab Emirates",
    "image_generation_prompt": "Generate an image of the iconic Burj Al Arab hotel in Dubai, showcasing its sail-shaped design and luxurious architecture.",
    "error": null
}


In [33]:
import json
json.loads(landmark_details.model_dump_json(indent=4))['error']

In [29]:
from src.models import ReplicateImageGenerator

api_token = ""
generator = ReplicateImageGenerator(api_token=api_token)

model = "black-forest-labs/flux-pro"
input_params = {
    "width": 1024,
    "height": 1024,
    "prompt": "Generate an image of the iconic Burj Al Arab hotel in Dubai, showcasing its sail-shaped design and luxurious architecture."

}
output_path = generator.generate_image(model, input_params)
print(f"Generated image path: {output_path}")

Image saved as generated_images/2c5d532ef5a64f47a8b21a0619253b78.jpg
Generated image path: generated_images/2c5d532ef5a64f47a8b21a0619253b78.jpg


In [48]:
import json


class LandmarkProcessor:
    """
    A class to handle both landmark information retrieval and image generation.
    """
    def __init__(self, api_key: str, replicate_api_token: str):
        """
        Initialize the LandmarkProcessor with API keys for LLMRetriever and ReplicateImageGenerator.
        """
        self.retriever = LLMRetriever(api_key)
        self.generator = ReplicateImageGenerator(api_token=replicate_api_token)

    def process_landmark(self, landmark_name: str) -> str:
        """
        Fetch landmark details and generate an image based on the retrieved details.

        Args:
            landmark_name (str): The name of the landmark.

        Returns:
            str: Path to the generated image.
        
        Raises:
            ValueError: If an error exists in the landmark retrieval response.
        """
        # Fetch landmark details
        landmark_details = self.retriever.fetch_landmark_details(landmark_name)

        # Convert landmark details to JSON
        landmark_json = json.loads(landmark_details.model_dump_json(indent=4))

       # Check for errors in the response
        if "error" in landmark_json and landmark_json["error"]:
            raise ValueError(f"Error fetching landmark details: {landmark_json['error']}")

        # Check for 'no_value' in more than one key
        no_value_count = sum(1 for key, value in landmark_json.items() if value == 'no_value')
        if no_value_count > 1:
            raise ValueError(f"Error fetching landmark details: Multiple fields contain 'no_value'.")

        # Build the image generation prompt using all relevant keys
        prompt_parts = [
            f"Description: {landmark_json.get('description', 'No description available.')}",
            f"Architects: {landmark_json.get('architects', 'No architects available.')}",
            f"Construction Year: {landmark_json.get('construction_year', 'No year available.')}",
            f"Materials: {landmark_json.get('materials', 'No materials available.')}",
            f"Interesting Facts: {landmark_json.get('interesting_facts', 'No facts available.')}",
            f"Dimensions: {landmark_json.get('dimensions', 'No dimensions available.')}",
            f"Construction Cost: {landmark_json.get('construction_cost', 'No cost available.')}",
            f"Cultural Significance: {landmark_json.get('cultural_significance', 'No significance available.')}",
            f"Geographical Location: {landmark_json.get('geographical_location', 'No location available.')}",
        ]

        # Add the main image generation prompt
        prompt_parts.append(
            f"Image Prompt: {landmark_json.get('image_generation_prompt', 'No prompt available.')}"
        )

        # Combine all parts into a single prompt
        full_prompt = "\n".join(prompt_parts)

        # Generate and save the image
        output_path = self.generator.generate_image(
            model="black-forest-labs/flux-pro",
            input_params={
                "width": 1024,
                "height": 1024,
                "prompt": full_prompt
            }
        )
        return output_path, landmark_details


#from PIL import Image


api_key = ""
replicate_api_token = ""

# Initialize the processor
processor = LandmarkProcessor(api_key=api_key, replicate_api_token=replicate_api_token)

# Call the LandmarkProcessor to generate the image and get JSON data
image_path, landmark_details = processor.process_landmark("eifel tower")


landmark_details

Image saved as generated_images/cb4cfad4c4814f9987eab95db9ed7aed.jpg


LandmarkResponse(description="The Eiffel Tower is an iron lattice tower located in Paris, France, built for the 1889 World's Fair. It was intended to be a temporary structure, but it became an instant icon of Paris and a symbol of French culture and engineering ingenuity.", architects='Gustave Eiffel, Maurice Koechlin, Émile Nouguier', construction_year='1889', materials='Iron lattice, Wrought iron', interesting_facts='The Eiffel Tower was originally intended to be a temporary structure, but it became an instant icon of Paris., It was the tallest man-made structure in the world when it was first built.', dimensions='Height: 324 meters (1,063 ft), Base dimensions: 125 meters (410 ft) square', construction_cost='Approximately 8 million French gold francs (around $1.5 million USD in 1889)', cultural_significance='The Eiffel Tower is a symbol of French culture and engineering ingenuity, and one of the most recognizable landmarks in the world.', geographical_location='Paris, France, on the 